Как ставить написано в документации и документации.

Предварительно проверьте, что у вас установлена Java 11.

neo4j-admin set-initial-password ANewPassword

В файле Neo4J.conf необходимо раскомментировать следующий ключ. dbms.security.auth_enabled=false Перезапускаем СУБД.

Теперь заходим в админку базы данных. http://127.0.0.1:7474/

In [94]:
from neo4j import GraphDatabase
from flask import Flask
from flask import url_for, render_template, request, redirect

import re

In [95]:
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"), encrypted=False)

In [365]:
app = Flask(__name__)

In [366]:
session = {}
session['username'] = None

In [367]:
@app.route('/home', methods=['GET', 'POST'])
def index():
    if session['username'] == None:
        return render_template('home.html')
    else:
        tasks = []
        importance = []
        statuses = []
        ids = []
        
        if request.method == "POST":
            det = request.form
            importance_list = []
            status_list = []
            if 'importance1' in det:
                importance_list.append(det['importance1'])
            if 'importance2' in det:
                importance_list.append(det['importance2'])
            if 'importance3' in det:
                importance_list.append(det['importance3'])
    
            if 'status1' in det:
                status_list.append(det['status1'])
            if 'status2' in det:
                status_list.append(det['status2'])
            if 'status3' in det:
                status_list.append(det['status3'])
           
            if importance_list == [] and status_list == []:
                c = driver.session().run("MATCH (f {nickname: $nickname})-[:HAS_TASK]->(t) return t.task_name, t.importance, t.status, t.deadline", nickname=session['username'])
            else:
                base_string = 'MATCH (f {nickname: $nickname})-[:HAS_TASK]->(t)'
                if importance_list or status_list:
                    base_string += ' WHERE ('
                if importance_list:
                    for i, element in enumerate(importance_list):
                        base_string = base_string + 't.importance = \'' + element + '\''
                        if i != len(importance_list) - 1:
                            base_string += ' OR '
                        else:
                            base_string += ')'
                if status_list:
                    if importance_list:
                        base_string += ' AND ('
                    for i, element in enumerate(status_list):
                        base_string = base_string + 't.status = \'' + element + '\''
                        if i != len(status_list) - 1:
                            base_string += ' OR '
                        else:
                            base_string += ')'
            base_string += ' RETURN t.task_name, t.importance, t.status, t.deadline'
            c = driver.session().run(base_string, nickname = session['username'])
        else:
            c = driver.session().run("MATCH (f {nickname: $nickname})-[:HAS_TASK]->(t) return t.task_name, t.importance, t.status, t.deadline", nickname=session['username'])
        data = []
        for i in c.data():
                elem = [i['t.task_name'], i['t.importance'], i['t.status'], i['t.deadline']]
                data.append(elem)
        return render_template('home_logged_in.html', data = data, username = session['username'])

In [368]:
@app.route('/signup', methods=['GET', 'POST'])
def sign_up():
    if request.method == "POST":
        username = request.form['username']
        password = request.form['password']
        c = driver.session().run("match (n {nickname: $nickname}) return n.nickname", nickname = username)
        if c.data():
            return render_template('user_exists.html')
        else:
            driver.session().run("CREATE (f:User {nickname: $nickname, password: $password, n_of_friends: 0, n_of_requests: 0})", nickname = username, password = password)
            return redirect('login')
    return render_template('sign_up.html')

In [369]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == "POST":
        username = request.form['username']
        password = request.form['password']
        c = driver.session().run("match (n {nickname: $nickname}) return n.nickname, n.password", nickname = username)
        result = c.data()
        if result:
            if result[0]['n.password'] == password:
                session['username'] = username
                return redirect('home')
            else:
                return render_template('wrong_password.html')
        else:
            return render_template('no_user.html')
    return render_template('login.html')

In [370]:
@app.route('/logout', methods=['GET', 'POST'])
def log_out():
    session['username'] = None
    return redirect('/login')

In [371]:
@app.route('/new_task', methods=['GET', 'POST'])
def new_task(): 
    if request.method == "POST":
        details = request.form
        
        c = driver.session().run('MATCH (f {nickname: $nickname})\
                                CREATE (t:Task {task_name: $task_name, importance: $importance, status: $status, deadline: $deadline, creator: $nickname})\
                                CREATE (f)-[:HAS_TASK]->(t)', nickname = session['username'], task_name = details['tasks'], importance = details['importance'], status = details['status'], deadline = details['deadline'])
        
        if details['friend'] != '':
            c = driver.session().run('MATCH (n {nickname: $nickname})-[:HAS_TASK]->(t{task_name: $task_name, importance: $importance, status: $status, deadline: $deadline, creator: $nickname})\
                                     MATCH (f {nickname: $friend_name}) \
                                     CREATE (t)-[:SEND_TASK_TO]->(f)', nickname = session['username'], friend_name = details['friend'], task_name = details['tasks'], importance = details['importance'], status = details['status'], deadline = details['deadline'])
        return redirect('home')
    
    c = driver.session().run("MATCH (f {nickname: $nickname})-[:FRIEND_OF]->(m) return m.nickname", nickname = session['username'])
    f_data = c.data()
    if f_data:
        friends = [result['m.nickname'] for result in f_data]
        friends.insert(0, '') 
    else:
        friends = []
    importance = ['не очень важно', 'важно', 'очень важно']
    status = ['не начато', 'в процессе', 'закончено']
    return render_template('new_task.html', status = status, importance = importance, friends=friends)

In [372]:
@app.route('/friends_tasks', methods=['GET', 'POST'])
def check_tasks():
    '''здесь должен глядеть задачи от друзей'''
    c = driver.session().run("MATCH (t)-[:SEND_TASK_TO]->(u{nickname: $nickname}) return t.task_name, t.importance, t.status, t.deadline, t.creator", nickname = session['username'])
    data = c.data()
    t_requests = []
    if data:
        t_requests = [[i['t.task_name'], i['t.importance'], i['t.status'], i['t.deadline'], i['t.creator']] for i in data]
        
    return render_template('tasks_from_fr.html', t_requests = t_requests)

In [373]:
@app.route('/reply_to_friends_tasks', methods = ["POST", "GET"])
def reply_to_task_request():
    if request.args.get('task_req'):
        friendname = str(request.args.get('nickname'))
        taskname = str(request.args.get('task_name'))
        importance_f = str(request.args.get('importance'))
        status_f = str(request.args.get('status'))
        deadline_f = str(request.args.get('deadline'))
        
        if request.args.get('task_req') == 'Принять':
            driver.session().run('MATCH (n {nickname: $friend_name})-[:HAS_TASK]->(t {task_name: $task_name, importance: $importance, status: $status, deadline: $deadline, creator: $friend_name})\
                                MATCH (f {nickname: $nickname})\
                                CREATE (f)-[:HAS_TASK]->(t)', nickname = session['username'], friend_name = friendname, task_name = taskname, importance = importance_f, status = status_f, deadline = deadline_f)

        driver.session().run("MATCH (t {task_name: $task_name, importance: $importance, status: $status, deadline: $deadline, creator: $creator})-[r:SEND_TASK_TO]->(m {nickname: $nickname})\
                            DELETE r", nickname = session['username'], creator = friendname, task_name = taskname, importance = importance_f, status = status_f, deadline = deadline_f)
        return redirect('/friends_tasks')
    else:
        return ('You should not be here')

In [374]:
@app.route('/update', methods=['GET', 'POST'])
def update_task():
    data = request.args.get('data').strip('\[\]')
    as_list = data.split(', ')
    d = [i.strip('\'') for i in as_list]
    if request.method == "POST":
        details = request.form
        
        driver.session().run('MATCH (f {nickname: $nickname})-[:HAS_TASK]->(t) WHERE t.task_name = $task_name AND t.importance = $importance AND t.status = $status\
                            SET t.importance = $new_importance, t.status = $new_status',
                            nickname = session['username'], task_name = d[0], importance = d[1], status = d[2],
                            new_importance = details['importance'], new_status = details['status'])
        return redirect('home')
    
    importance = ['не очень важно', 'важно', 'очень важно']
    status = ['не начато', 'в процессе', 'закончено']
    return render_template('update_task.html', status = status, importance = importance, task = d[0])

In [375]:
@app.route('/friends', methods = ['POST', 'GET'])
def friends():
    c = driver.session().run("MATCH (f {nickname: $nickname})-[:FRIEND_OF]->(m) return m.nickname", nickname = session['username'])
    data = c.data()
    if data:
        friends_usernames = [result['m.nickname'] for result in data]
    else:
        friends_usernames = []
        
    c = driver.session().run("MATCH (f {nickname: $nickname})<-[:SENT_REQUEST_TO]-(m) return m.nickname", nickname = session['username'])
    data = c.data()
    if data:
        requests = [result['m.nickname'] for result in data]
    else:
        requests = []
    return render_template('friends.html', usernames = friends_usernames, requests = requests)

In [376]:
@app.route('/send_request', methods = ["POST", "GET"])
def send_request():
    if request.args.get('username'):
        username = request.args.get('username')
        if username == session['username']:
            return redirect('/home')
        res = driver.session().run("MATCH (f {nickname: $nickname}) RETURN f.nickname", nickname = username)
        if res.data():
            driver.session().run("MATCH (f {nickname: $nickname1})\
                                MATCH (m {nickname:$nickname2})\
                                CREATE (f)-[:SENT_REQUEST_TO]->(m)",
                                nickname1 = session['username'], nickname2 = username)
            return redirect('/friends')
        else:
            return render_template('no_user.html')
    else:    
    
        return ('Не получил юзернейм.')

In [377]:
@app.route('/reply_to_request', methods = ["POST", "GET"])
def reply_to_request():
    if request.args.get('friend_req'):
        username = str(request.args.get('nickname'))
        with open ('m.txt', 'w') as f:
            f.write(username)
        if request.args.get('friend_req') == 'Принять':
            driver.session().run("MATCH (f {nickname: $nickname1})\
                                MATCH (m {nickname:$nickname2})\
                                CREATE (f)-[:FRIEND_OF]->(m)\
                                CREATE (f)<-[:FRIEND_OF]-(m)",
                                nickname1 = session['username'], nickname2 = username)
        driver.session().run("MATCH (f {nickname: $nickname1})<-[r:SENT_REQUEST_TO]-(m {nickname: $nickname2})\
                            DELETE r",
                            nickname1 = session['username'], nickname2 = username)
        return redirect('/friends')
    else:
        return ('You should not be here')

In [378]:
@app.route('/submit_action', methods = ["POST", "GET"])
def submit_action():
    nickname = session['username']
    if request.args.get('action')  and nickname != None:
        action = str(request.args.get('action'))
        if action == 'delete_friend':
            friendname = str(request.args.get('friendname'))
            question = "Вы действительно хотите удалить " + friendname + " из друзей?"
            return render_template('submit_fr_deletion.html', friendname = friendname, question = question) 
        elif action == "delete_myself":
            question = "Введите пароль, чтобы удалить Ваш аккаунт."
            return render_template('submit_myself_deletion.html', question = question) 
        elif action == "delete_task":
            data = request.args.get('data').strip('\[\]')
            as_list = data.split(', ')
            d = [i.strip('\'') for i in as_list]
            question = "Вы действительно хотите удалить задачу \"" + d[0] + "\"?"
            return render_template('submit_task_deletion.html', question = question, data = d) 
    else:
        return ('You should not be here')

In [379]:
@app.route('/delete_task', methods=['POST', 'GET'])
def delete():
    if request.args.get('data'):
        if request.args.get('task_del') == 'Да':
            data = request.args.get('data').strip('\[\]')
            as_list = data.split(', ')
            d = [i.strip('\'') for i in as_list]
            c = driver.session().run('MATCH (f {nickname: $nickname})-[:HAS_TASK]->(t) WHERE t.task_name = $task_name AND t.importance = $importance AND t.status = $status\
                                    DETACH DELETE t',
                                    nickname = session['username'], task_name = d[0], importance = d[1], status = d[2])
        return redirect('home')
    else:
        return ('You should not be here')

In [380]:
@app.route('/delete_friend', methods = ["POST", "GET"])
def delete_friend():
    if request.args.get('friend_del'):
        friendname = str(request.args.get('friendname'))
        if request.args.get('friend_del') == 'Да':
            driver.session().run("MATCH (f {nickname: $nickname})<-[r:FRIEND_OF]->(m {nickname: $friend}) DELETE r",
                                 nickname = session['username'], friend = friendname)
        return redirect('/friends')
    else:
        return ('You should not be here')

In [381]:
@app.route('/delete_myself', methods = ["POST", "GET"])
def delete_myself():    
    if request.args.get('iwannadie'):
        nickname = session['username']
        password = str(request.args.get('password'))
        c = driver.session().run("match (n {nickname: $nickname}) return n.nickname, n.password", nickname = nickname)
        result = c.data()
        if result:
            if result[0]['n.password'] == password:
                driver.session().run("MATCH (f {nickname: $nickname}) DETACH DELETE (f)" , nickname = nickname)
                session['username'] = None
                return redirect('/login')
            else:
                return render_template('wrong_password.html')
        else:
            return redirect('/home')
    else:
        return ('You should not be here')

In [382]:
@app.route('/<username>', methods = ["POST", "GET"])
def profile(username):
    c = driver.session().run("MATCH (n {nickname: $nickname}) return n.nickname", nickname = username)
    if c.data():
        t = driver.session().run("MATCH (f {nickname: $nickname})-[:HAS_TASK]->(t) return t.task_name, t.importance, t.status, t.deadline", nickname=username)
        data = []
        for i in t.data():
            elem = [i['t.task_name'], i['t.importance'], i['t.status'], i['t.deadline']]
            data.append(elem)
        
        if session['username']:
            t = driver.session().run("MATCH (f {nickname: $nickname1})-[:FRIEND_OF]->(m {nickname: $nickname2}) RETURN f", nickname1 = session['username'], nickname2 = username)
            if t.data():
                friend_str = 'Вы дружите с этим пользователем.'
            else:
                t = driver.session().run("Match path = shortestpath( (f:User {nickname:$nickname1}) –[r:FRIEND_OF *]->(p:User {nickname:$nickname2})) return length(path) as len", nickname1 = session['username'], nickname2 = username)
                data_path = t.data()
                if not data_path:
                    num = 0
                else:
                    num = data_path[0]['len']
                if (num % 100 >= 11 and num % 100 <= 19) or (num % 10 >= 5 and num % 10 <= 9) or (num % 10 == 0):
                    handshakes = ' рукопожатий.'
                elif num % 10 == 1:
                    handshakes = ' рукопожатие.'
                else:
                    handshakes = ' рукопожатия.'
                friend_str = 'Вы связаны с этим пользователем через ' + str(num) + handshakes
        else:
            friend_str = 'Войдите в систему, чтобы узнать, через сколько рукопожатий вы связаны с этим пользователем.'
        return render_template('user.html', username = username, data = data, friend_str = friend_str)
    else:
        return render_template('no_user.html')

In [383]:
if __name__ == '__main__':
    app.run(debug=False)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Apr/2020 00:36:46] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 00:36:46] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 00:36:59] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [04/Apr/2020 00:36:59] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 00:36:59] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 00:37:09] "GET /friends HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 00:37:09] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 00:37:12] "GET /ejouny HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 00:37:12] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 00:37:17] "GET /friends HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 00:37:17] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 00:37:22] "GET /reply_to_request?nickname=ejouny&friend_req=%D0%9F%D1%80%D0%B8%D0%BD%D1%8F%D1%82%D1%8C HTTP/1.1" 302 -
127.0.0.1 - - [04/Apr/2020 00:37:23] "GET /f

In [307]:
c = driver.session().run("Match path = shortestpath( (f:User {nickname:$nickname1}) –[r:FRIEND_OF *]->(p:User {nickname:$nickname2})) return length(path) as len", nickname1 = 'fishowl', nickname2 = 'kolbasa')

In [101]:
d = c.data()

In [103]:
print(d)

[]


In [87]:
len(d[0])

1